<center style="font-size: 26px"> <b>Проверка уровня знаний Python</b></center>

In [1]:
# обеспечиваем совместимость с Python 2 и 3
from __future__ import (absolute_import, division, print_function, unicode_literals)

# отключаем предупреждения дистрибутива Anaconda
import warnings
warnings.simplefilter('ignore')

**Описание датасета:**   
[Датасет вакансий с платформы HH.ru на Kaggle.com](https://www.kaggle.com/datasets/pavfedotov/heaadhunter-vacancies?resource=download&select=df2021-08-03.csv) 

Нужный файл - df2021-08-03.csv.

- *vacancy* - наименование вакансии
- *url* - ссылка на вакансию
- *created* - дата и время создания
- *has_test* - наличие тестового задания в вакансии
- *salary_from* - нижняя граница значения заработной платы
- *salary_to* - верхняя граница значения заработной платы
- *currency* - валюта заработной платы
- *experience* - требуемый опыт
- *schedule* - тип рабочего графика
- *skills* - требуемые навыки
- *employer* - наименование работодателя
- *area* - наименование города
- *description* - описание вакансии

In [2]:
import pandas as pd
import numpy as np

#### Загрузите датасет

In [3]:
# ВАШ КОД ЗДЕСЬ
#в явном виде зададим формат datetime для колонки с датами
df = pd.read_csv('df2021-08-03.csv', parse_dates=['created'])
df.head()

,vacancy,url,created,has_test,salary_from,salary_to,currency,experience,schedule,skills,employer,area,description
0,Backend/Full-stack developer (python),https://hh.ru/applicant/vacancy_response?vacan...,2021-08-01 13:02:48+03:00,False,120000.0,NaN,RUR,От 1 года до 3 лет,Гибкий график,Python;PostgreSQL;Linux;Flask;,ATI.SU,Санкт-Петербург,Привет! Мы ATI.SU ― крупнейшая в России компан...
1,Бэкенд-разработчик (Python) / Middle Python / ...,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-09 08:13:01+03:00,False,150000.0,220000.0,RUR,От 3 до 6 лет,Удаленная работа,Python;Git;MongoDB;Redis;Design Patterns;Flask;,"ЮТэйр, Авиакомпания",Киров (Кировская область),Utair - российская авиакомпания. Мы летаем по ...
2,Fullstack Middle Python (Django) / VueJS Dev,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-21 15:24:16+03:00,False,NaN,NaN,NaN,От 1 года до 3 лет,Полный день,Python;Git;Django Framework;PostgreSQL;Linux;V...,КРОК,Москва,"В команду, которая занимается разработкой инст..."
3,Python Developer,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-28 12:57:57+03:00,False,NaN,NaN,NaN,От 3 до 6 лет,Удаленная работа,Python;Django Framework;CSS;,Mad Devs,Краснодар,Mad Devs - full stack команда для разработки и...
4,QA Automation (Python),https://hh.ru/applicant/vacancy_response?vacan...,2021-07-31 17:10:49+03:00,False,NaN,NaN,NaN,От 1 года до 3 лет,Удаленная работа,Python;Pytest;Ansible;Selenium;,Аренадата Софтвер,Москва,"Arenadata — динамично развивающаяся компания, ..."


In [4]:
#посмторим на типы данных в колонках
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1700 entries, 0 to 1699
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype                                
---  ------       --------------  -----                                
 0   vacancy      1700 non-null   object                               
 1   url          1700 non-null   object                               
 2   created      1700 non-null   datetime64[ns, pytz.FixedOffset(180)]
 3   has_test     1700 non-null   bool                                 
 4   salary_from  538 non-null    float64                              
 5   salary_to    433 non-null    float64                              
 6   currency     613 non-null    object                               
 7   experience   1700 non-null   object                               
 8   schedule     1700 non-null   object                               
 9   skills       1490 non-null   object                               
 10  employer     1700 non-nu

In [5]:
df.shape

(1700, 13)

In [6]:
#посмотрим, есть ли NaN-значения в каких-либо колонках
df.isna().any()

vacancy        False
url            False
created        False
has_test       False
salary_from     True
salary_to       True
currency        True
experience     False
schedule       False
skills          True
employer       False
area           False
description    False
dtype: bool

In [7]:
#заполнять пропуски я буду чуть позже - для этого сначала создам новые колонки specialization и vacancy_level

In [8]:
df.vacancy.unique()

array(['Backend/Full-stack developer (python)',
       'Бэкенд-разработчик (Python) / Middle Python / Python Developer (Remote)',
       'Fullstack Middle Python (Django) / VueJS Dev', 'Python Developer',
       'QA Automation (Python)', 'Data Engineer / Python Developer',
       'Python developer', 'Ведущий Python разработчик',
       'Разработчик Python', 'Python developer with AWS expertise',
       'Middle Python developer (Москва)',
       'Python developer / python программист (middle+/senior)',
       'Audio\xa0Programmer (С++/ Python)',
       'Middle Python Developer for CloudLinux OS [Remote]',
       'Golang/Python разработчик', 'Программист Python',
       'Middle Backend Developer (Web app, Python, FastAPI) / Удаленно',
       'Разработчик Python (Django/Flask)',
       'Python Team Lead/Senior Python Developer',
       'Python Software Engineer', 'Python Test Developer',
       'Senior QA Automation Engineer (Python)',
       'Backend разработчик (Python)', 'Middle Python

In [9]:
def classify_vacancy(vacancy):
    v = vacancy.lower()
    if any(x in v for x in ['qa', 'sdet', 'автотест', 'тестирование', 'test developer', 'по автоматическому тестированию']):
        return 'QA/Тестирование'
    elif 'fullstack' in v or 'full stack'  in v or 'full-stack' in v:
        return 'Fullstack'
    elif 'backend' in v or 'бэкенд' in v or 'back-end' in v or 'software engineer' in v:
        return 'Backend'
    elif any(x in v for x in ['data scientist', 'machine learning', 'ml']):
        return 'Data Science/ML'
    elif 'data engineer' in v or 'etl' in v or 'dwh' in v:
        return 'Data Engineer'
    elif 'devops' in v or 'sre' in v:
        return 'DevOps/SRE'
    elif any(x in v for x in ['преподаватель', 'наставник', 'tutor', 'teacher']):
        return 'Преподавание'
    elif 'аналитик' in v or 'analyst' in v:
        return 'Аналитика'
    elif 'big data' in v or 'spark' in v or 'hadoop' in v:
        return 'Big Data'
    elif any(x in v for x in ['django', 'flask', 'fastapi']):
        return 'Python Web Developer'
    elif 'python' in v and any(x in v for x in ['developer', 'python-разработчик', 'программист-разработчик', 'programmer', 'программист', 'инженер-программист', 'разработчик']):
        return 'Python Developer (общий профиль)'
    else:
        return 'Другое'

df['specialization'] = df['vacancy'].apply(classify_vacancy)


In [10]:
df.experience.unique()

array(['От 1 года до 3 лет', 'От 3 до 6 лет', 'Нет опыта', 'Более 6 лет'],
      dtype=object)

In [11]:
def vacancy_level(data):
    if data == 'От 1 года до 3 лет':
        return 'junior'
    elif data == 'От 3 до 6 лет':
        return 'middle'
    elif data == 'Более 6 лет':
        return 'senior'
    else:
        return 'intern'

df['vacancy_level'] = df['experience'].apply(vacancy_level)

In [12]:
df.head(30)

,vacancy,url,created,has_test,salary_from,salary_to,currency,experience,schedule,skills,employer,area,description,specialization,vacancy_level
0,Backend/Full-stack developer (python),https://hh.ru/applicant/vacancy_response?vacan...,2021-08-01 13:02:48+03:00,False,120000.0,NaN,RUR,От 1 года до 3 лет,Гибкий график,Python;PostgreSQL;Linux;Flask;,ATI.SU,Санкт-Петербург,Привет! Мы ATI.SU ― крупнейшая в России компан...,Fullstack,junior
1,Бэкенд-разработчик (Python) / Middle Python / ...,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-09 08:13:01+03:00,False,150000.0,220000.0,RUR,От 3 до 6 лет,Удаленная работа,Python;Git;MongoDB;Redis;Design Patterns;Flask;,"ЮТэйр, Авиакомпания",Киров (Кировская область),Utair - российская авиакомпания. Мы летаем по ...,Backend,middle
2,Fullstack Middle Python (Django) / VueJS Dev,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-21 15:24:16+03:00,False,NaN,NaN,NaN,От 1 года до 3 лет,Полный день,Python;Git;Django Framework;PostgreSQL;Linux;V...,КРОК,Москва,"В команду, которая занимается разработкой инст...",Fullstack,junior
3,Python Developer,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-28 12:57:57+03:00,False,NaN,NaN,NaN,От 3 до 6 лет,Удаленная работа,Python;Django Framework;CSS;,Mad Devs,Краснодар,Mad Devs - full stack команда для разработки и...,Python Developer (общий профиль),middle
4,QA Automation (Python),https://hh.ru/applicant/vacancy_response?vacan...,2021-07-31 17:10:49+03:00,False,NaN,NaN,NaN,От 1 года до 3 лет,Удаленная работа,Python;Pytest;Ansible;Selenium;,Аренадата Софтвер,Москва,"Arenadata — динамично развивающаяся компания, ...",QA/Тестирование,junior
5,Data Engineer / Python Developer,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-28 14:58:53+03:00,False,NaN,NaN,NaN,От 1 года до 3 лет,Полный день,NaN,Devim,Санкт-Петербург,Привет! На связи Devim. Для начала – немного о...,Data Engineer,junior
6,Python developer,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-12 12:18:41+03:00,False,NaN,NaN,NaN,Нет опыта,Полный день,Python;Django Framework;PostgreSQL;Linux;,Factory5,Казань,О компании. Factory5 (группа Ctrl2GO) — разраб...,Python Developer (общий профиль),intern
7,Ведущий Python разработчик,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-06 23:46:27+03:00,False,200000.0,350000.0,RUR,Более 6 лет,Полный день,Python;Node.js;HTTP;Git;TCP/IP;JavaScript;PHP;...,ODDSCORP,Москва,"Мы - команда проекта ODDSCORP, небольшая групп...",Python Developer (общий профиль),senior
8,Разработчик Python,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-27 09:59:22+03:00,False,90000.0,180000.0,RUR,От 1 года до 3 лет,Гибкий график,Python;MySQL;Базы данных;SQL;Linux;Django Fram...,The Head,Екатеринбург,19 лет The Head создает и развивает IT-инфраст...,Python Developer (общий профиль),junior
9,Python developer with AWS expertise,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-13 11:06:01+03:00,False,NaN,NaN,NaN,Более 6 лет,Гибкий график,Python;Cloud technologies;AWS;SaaS;Git;Jenkins...,Deutsche Telekom IT Solutions (ex.T-Systems),Санкт-Петербург,Telekom IT is an integral part of Deutsche Tel...,Python Developer (общий профиль),senior


In [13]:
df.vacancy_level.unique()

array(['junior', 'middle', 'intern', 'senior'], dtype=object)

#### 1. Изучите методы для сбора данных курсов валют в [API Центрального банка Российской Федерации](https://www.cbr.ru/development/SXML/) . Используя запросы к API, приведите значения заработной платы к рублям одним из вариантов: 1) простой вариант - на текущую дату  2) вариант посложнее - на дату создания (created).

In [14]:
# ВАШ КОД ЗДЕСЬ

df.currency.unique()


array(['RUR', nan, 'EUR', 'USD'], dtype=object)

In [15]:
import requests
import pandas as pd
from datetime import datetime


def get_currency_rate(date, currency):
    if currency == 'RUR' or currency == 'RUB':
        return 1.0  #для рублей курс всегда 1
    
    date_str = date.strftime('%d/%m/%Y')
    url = f'https://www.cbr.ru/scripts/XML_daily.asp?date_req={date_str}'
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        
        #парсим XML ответа
        from xml.etree import ElementTree as ET
        root = ET.fromstring(response.content)
        
        for valute in root.findall('Valute'):
            if valute.find('CharCode').text == currency:
                value = float(valute.find('Value').text.replace(',', '.'))
                nominal = int(valute.find('Nominal').text)
                return value / nominal
                
        print(f"Курс {currency} на {date_str} не найден")
        return None
    except Exception as e:
        print(f"Ошибка при запросе курса {currency}: {e}")
        return None

#функция для конвертации зарплаты в рубли
def convert_salary(row):
    if pd.isna(row['currency']):
        return None, None
    
    
    currency = 'USD' if row['currency'] == 'USD' else \
               'EUR' if row['currency'] == 'EUR' else \
               'RUB'
    
    try:
        date = pd.to_datetime(row['created'])
        rate = get_currency_rate(date, currency)
        
        if rate is None:
            return None, None
        
       
        salary_from = row['salary_from'] * rate if not pd.isna(row['salary_from']) else None
        salary_to = row['salary_to'] * rate if not pd.isna(row['salary_to']) else None
        
        return salary_from, salary_to
    except Exception as e:
        print(f"Ошибка в строке {row.name}: {e}")
        return None, None


df[['salary_from_rub', 'salary_to_rub']] = df.apply(convert_salary, axis=1, result_type='expand')


print("Первые 5 строк после конвертации:")
print(df[['currency', 'salary_from', 'salary_to', 'salary_from_rub', 'salary_to_rub']].head())

Первые 5 строк после конвертации:
  currency  salary_from  salary_to  salary_from_rub  salary_to_rub
0      RUR     120000.0        NaN         120000.0            NaN
1      RUR     150000.0   220000.0         150000.0       220000.0
2      NaN          NaN        NaN              NaN            NaN
3      NaN          NaN        NaN              NaN            NaN
4      NaN          NaN        NaN              NaN            NaN


In [16]:
#далее будут использоваться колонки salary_from_rub и salary_to_rub

In [17]:
df[df['currency'].isna()].describe()

,salary_from,salary_to,salary_from_rub,salary_to_rub
count,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN


In [18]:
#исходя из этих описательных статистик видно, что имеются в виду рублевые зарплаты (особенно см на мин знач)

In [19]:
#для сравнения порядка чисел то же самое по евровой зарплате
df[df['currency'] == 'EUR'].describe()

,salary_from,salary_to,salary_from_rub,salary_to_rub
count,6.000000,11.000000,6.000000,11.000000
mean,2616.666667,3754.545455,227731.831667,326554.649091
std,702.614166,694.785775,60994.492564,60843.608879
min,1800.000000,3000.000000,157170.780000,260973.900000
25%,2125.000000,3500.000000,184856.512500,303957.850000
50%,2500.000000,3500.000000,217478.250000,303957.850000
75%,3175.000000,3750.000000,276228.232500,325991.800000
max,3500.000000,5500.000000,304469.550000,478452.150000


In [20]:
#поэтому пропуски в currency заполняем на RUR

In [21]:
df['currency'] = df['currency'].fillna('RUR')

In [22]:
df.isna().any()

vacancy            False
url                False
created            False
has_test           False
salary_from         True
salary_to           True
currency           False
experience         False
schedule           False
skills              True
employer           False
area               False
description        False
specialization     False
vacancy_level      False
salary_from_rub     True
salary_to_rub       True
dtype: bool

In [23]:
#теперь перейдем к заполнению пропущенных значений

Распределение зарплат почти всегда асимметричное, с длинным хвостом в сторону высоких значений, при этом обычно есть выбросы: иногда единичные вакансии предлагают сильно завышенные или заниженные оклады.

Поэтому наиболее репрезентативна медианная зарплата.

In [24]:
group_by = df.groupby(['specialization', 'vacancy_level', 'area']).agg({'salary_from_rub': 'median', 'salary_to_rub': 'median'}) \
    .dropna() \
    .sort_values(['salary_from_rub', 'salary_to_rub'], ascending=False)
group_by

salary_from_rub  \
specialization                   vacancy_level area                                         
Data Science/ML                  middle        Москва                           250000.00   
QA/Тестирование                  middle        Москва                           250000.00   
Python Developer (общий профиль) senior        Санкт-Петербург                  233649.92   
Data Science/ML                  junior        Санкт-Петербург                  219416.40   
Fullstack                        intern        Москва                           217478.25   
...                                                                                   ...   
Python Developer (общий профиль) junior        Петропавловск-Камчатский          25000.00   
                                 intern        Шахты                             25000.00   
Преподавание                     junior        Пятигорск                         20000.00   
                                 intern        Северск (Томская область)         16000.00   
                                               Томск                             16000.00   

                                                                          salary_to_rub  
specialization                   vacancy_level area                                      
Data Science/ML                  middle        Москва                         275000.00  
QA/Тестирование                  middle        Москва                         210000.00  
Python Developer (общий профиль) senior        Санкт-Петербург                306665.52  
Data Science/ML                  junior        Санкт-Петербург                365694.00  
Fullstack                        intern        Москва                         260973.90  
...                                                                                 ...  
Python Developer (общий профиль) junior        Петропавловск-Камчатский        50000.00  
                                 intern        Шахты                           40000.00  
Преподавание                     junior        Пятигорск                       25000.00  
                                 intern        Северск (Томская область)       32000.00  
                                               Томск                           32000.00  

[180 rows x 2 columns]

In [25]:
# преобразуем group_by в словари для быстрого доступа
median_from_dict = group_by['salary_from_rub'].to_dict()  
median_to_dict = group_by['salary_to_rub'].to_dict()

#функция для заполнения пропусков
def fill_salaries(row):
  
    if pd.isna(row['salary_from_rub']):
        key = (row['specialization'], row['vacancy_level'], row['area'])
        if key in median_from_dict:
            row['salary_from_rub'] = median_from_dict[key]
    
   
    if pd.isna(row['salary_to_rub']):
        key = (row['specialization'], row['vacancy_level'], row['area'])
        if key in median_to_dict:
            row['salary_to_rub'] = median_to_dict[key]
    
    return row

mask = df['salary_from_rub'].isna() | df['salary_to_rub'].isna()
df[mask] = df[mask].apply(fill_salaries, axis=1)


print(f"Осталось пропусков в salary_from_rub: {df['salary_from_rub'].isna().sum()}")
print(f"Осталось пропусков в salary_to_rub: {df['salary_to_rub'].isna().sum()}")

Осталось пропусков в salary_from_rub: 274
Осталось пропусков в salary_to_rub: 291


In [26]:
#для оставшихся пропусков делаем более простую группировку

In [27]:
group_by_new = df.groupby(['specialization', 'vacancy_level']).agg({'salary_from_rub': 'median', 'salary_to_rub': 'median'}) \
    .dropna() \
    .sort_values(['salary_from_rub', 'salary_to_rub'], ascending=False)
group_by_new

salary_from_rub  salary_to_rub
specialization                   vacancy_level                                
Data Science/ML                  middle               250000.00      275000.00
QA/Тестирование                  middle               250000.00      210000.00
Fullstack                        middle               200000.00      500000.00
                                 senior               200000.00      500000.00
Python Web Developer             senior               200000.00      350000.00
Python Developer (общий профиль) senior               200000.00      306665.52
Другое                           middle               200000.00      300000.00
Backend                          middle               190000.00      250000.00
Python Developer (общий профиль) middle               180000.00      240000.00
Backend                          senior               150000.00      500000.00
                                 junior               140000.00      200000.00
Python Web Developer             middle               130000.00      170000.00
Аналитика                        junior               120000.00      240000.00
Data Engineer                    intern               120000.00      150000.00
Backend                          intern               110649.45      304018.40
Fullstack                        junior               105000.00      250000.00
Другое                           junior               100000.00      210000.00
Python Web Developer             junior               100000.00      190000.00
Python Developer (общий профиль) junior               100000.00      180000.00
QA/Тестирование                  junior               100000.00      170000.00
Fullstack                        intern               100000.00      167986.95
Data Engineer                    junior               100000.00      140000.00
Python Web Developer             intern                95000.00       80000.00
Python Developer (общий профиль) intern                60000.00      140000.00
Data Science/ML                  junior                50000.00      150000.00
Другое                           intern                50000.00       85000.00
Преподавание                     junior                20000.00       52500.00
                                 intern                16000.00       32000.00

In [28]:
#создаем словари из новой группировки
median_from_dict_new = group_by_new['salary_from_rub'].to_dict()  
median_to_dict_new = group_by_new['salary_to_rub'].to_dict()

#функция для заполнения пропусков
def fill_remaining_salaries(row):

    if pd.isna(row['salary_from_rub']):
        key = (row['specialization'], row['vacancy_level'])
        if key in median_from_dict_new:
            row['salary_from_rub'] = median_from_dict_new[key]
    

    if pd.isna(row['salary_to_rub']):
        key = (row['specialization'], row['vacancy_level'])
        if key in median_to_dict_new:
            row['salary_to_rub'] = median_to_dict_new[key]
    
    return row


mask_remaining = df['salary_from_rub'].isna() | df['salary_to_rub'].isna()
df[mask_remaining] = df[mask_remaining].apply(fill_remaining_salaries, axis=1)


print(f"Осталось пропусков в salary_from_rub: {df['salary_from_rub'].isna().sum()}")
print(f"Осталось пропусков в salary_to_rub: {df['salary_to_rub'].isna().sum()}")

Осталось пропусков в salary_from_rub: 49
Осталось пропусков в salary_to_rub: 51


In [29]:
#финальное заполнение глобальными медианами

In [30]:
group_by_new1 = df.groupby('specialization').agg({'salary_from_rub': 'median', 'salary_to_rub': 'median'}) \
    .dropna() \
    .sort_values(['salary_from_rub', 'salary_to_rub'], ascending=False)
group_by_new1

,salary_from_rub,salary_to_rub
specialization,,
Data Science/ML,250000.0,275000.00
Fullstack,175000.0,260212.05
Backend,140000.0,220000.00
Python Developer (общий профиль),130000.0,200000.00
QA/Тестирование,125000.0,210000.00
Аналитика,120000.0,240000.00
Python Web Developer,115000.0,190000.00
Другое,100000.0,210000.00
Data Engineer,100000.0,140000.00


In [31]:
#создаем словари из простой группировки по специализации
median_from_dict_simple = group_by_new1['salary_from_rub'].to_dict()
median_to_dict_simple = group_by_new1['salary_to_rub'].to_dict()

#функция для заполнения пропусков
def fill_by_specialization(row):

    if pd.isna(row['salary_from_rub']) and pd.notna(row['specialization']):
        if row['specialization'] in median_from_dict_simple:
            row['salary_from_rub'] = median_from_dict_simple[row['specialization']]
    

    if pd.isna(row['salary_to_rub']) and pd.notna(row['specialization']):
        if row['specialization'] in median_to_dict_simple:
            row['salary_to_rub'] = median_to_dict_simple[row['specialization']]
    
    return row

mask_simple = df['salary_from_rub'].isna() | df['salary_to_rub'].isna()
df[mask_simple] = df[mask_simple].apply(fill_by_specialization, axis=1)

#финальное заполнение глобальными медианами
global_median_from = df['salary_from_rub'].median()
global_median_to = df['salary_to_rub'].median()

df['salary_from_rub'].fillna(global_median_from, inplace=True)
df['salary_to_rub'].fillna(global_median_to, inplace=True)


print("\nРезультат заполнения:")
print(f"Осталось пропусков в salary_from_rub: {df['salary_from_rub'].isna().sum()}")
print(f"Осталось пропусков в salary_to_rub: {df['salary_to_rub'].isna().sum()}\n")



Результат заполнения:
Осталось пропусков в salary_from_rub: 0
Осталось пропусков в salary_to_rub: 0



In [32]:
df.isna().any()

vacancy            False
url                False
created            False
has_test           False
salary_from         True
salary_to           True
currency           False
experience         False
schedule           False
skills              True
employer           False
area               False
description        False
specialization     False
vacancy_level      False
salary_from_rub    False
salary_to_rub      False
dtype: bool

In [33]:
#заполним пропуски в skills модами

In [34]:
df.skills

0                          Python;PostgreSQL;Linux;Flask;
1         Python;Git;MongoDB;Redis;Design Patterns;Flask;
2       Python;Git;Django Framework;PostgreSQL;Linux;V...
3                            Python;Django Framework;CSS;
4                         Python;Pytest;Ansible;Selenium;
                              ...                        
1695                            Python;Git;Linux;iOS;C++;
1696                                                  NaN
1697    Python;Git;Linux;C++;x86;Windows;Unix;Virtuali...
1698    Git;PostgreSQL;Python;Blockchain;Crypto;React ...
1699    Java Servlets;Java;JavaScript;Spring Framework...
Name: skills, Length: 1700, dtype: object

In [35]:
skills_mode = df.groupby('specialization')['skills'].agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan)
skills_mode

specialization
Backend                               Python;Git;MongoDB;Redis;Design Patterns;Flask;
Big Data                                                     Python;Linux;PostgreSQL;
Data Engineer                       ETL;vertica;SQL;Python;Pandas;DWH;Olap (online...
Data Science/ML                     Python;SQL;MS SQL Server;ML;Анализ данных;Мате...
DevOps/SRE                                          Python;Linux;MySQL;MongoDB;Nginx;
Fullstack                           Python;JavaScript;Английский язык;AWS;Работа с...
Python Developer (общий профиль)                                              Python;
Python Web Developer                Python;Django Framework;PostgreSQL;SQL;ООП;Doc...
QA/Тестирование                     Django Framework;Публичные выступления;Pytest;...
Аналитика                                                               pandas;numpy;
Другое                                                                        Python;
Преподавание                          J

In [36]:
def fill_skills(row):
    if pd.isna(row['skills']) and pd.notna(row['specialization']):
        if row['specialization'] in skills_mode:
            return skills_mode[row['specialization']]
    return row['skills']


df['skills'] = df.apply(fill_skills, axis=1)
print(f"Осталось пропусков в skills: {df['skills'].isna().sum()}")

Осталось пропусков в skills: 0


In [37]:
df.isna().any()

vacancy            False
url                False
created            False
has_test           False
salary_from         True
salary_to           True
currency           False
experience         False
schedule           False
skills             False
employer           False
area               False
description        False
specialization     False
vacancy_level      False
salary_from_rub    False
salary_to_rub      False
dtype: bool

In [38]:
#за ненадобностью удалим колонки salary_from и salary_to, чтобы они не создавали NaN-значений
df = df.drop(['salary_from', 'salary_to'], axis=1)
df.isna().any()

vacancy            False
url                False
created            False
has_test           False
currency           False
experience         False
schedule           False
skills             False
employer           False
area               False
description        False
specialization     False
vacancy_level      False
salary_from_rub    False
salary_to_rub      False
dtype: bool

In [39]:
df.isna().any().sum()

0

#### 2. Найдите вакансию с самой высокой заработной платой, где подойдет 4 года опыта работы и не нужно проходить тестовое задание.

In [40]:
df.head()

,vacancy,url,created,has_test,currency,experience,schedule,skills,employer,area,description,specialization,vacancy_level,salary_from_rub,salary_to_rub
0,Backend/Full-stack developer (python),https://hh.ru/applicant/vacancy_response?vacan...,2021-08-01 13:02:48+03:00,False,RUR,От 1 года до 3 лет,Гибкий график,Python;PostgreSQL;Linux;Flask;,ATI.SU,Санкт-Петербург,Привет! Мы ATI.SU ― крупнейшая в России компан...,Fullstack,junior,120000.0,120000.0
1,Бэкенд-разработчик (Python) / Middle Python / ...,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-09 08:13:01+03:00,False,RUR,От 3 до 6 лет,Удаленная работа,Python;Git;MongoDB;Redis;Design Patterns;Flask;,"ЮТэйр, Авиакомпания",Киров (Кировская область),Utair - российская авиакомпания. Мы летаем по ...,Backend,middle,150000.0,220000.0
2,Fullstack Middle Python (Django) / VueJS Dev,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-21 15:24:16+03:00,False,RUR,От 1 года до 3 лет,Полный день,Python;Git;Django Framework;PostgreSQL;Linux;V...,КРОК,Москва,"В команду, которая занимается разработкой инст...",Fullstack,junior,175000.0,350000.0
3,Python Developer,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-28 12:57:57+03:00,False,RUR,От 3 до 6 лет,Удаленная работа,Python;Django Framework;CSS;,Mad Devs,Краснодар,Mad Devs - full stack команда для разработки и...,Python Developer (общий профиль),middle,100000.0,260000.0
4,QA Automation (Python),https://hh.ru/applicant/vacancy_response?vacan...,2021-07-31 17:10:49+03:00,False,RUR,От 1 года до 3 лет,Удаленная работа,Python;Pytest;Ansible;Selenium;,Аренадата Софтвер,Москва,"Arenadata — динамично развивающаяся компания, ...",QA/Тестирование,junior,100000.0,170000.0


In [41]:
df.experience.unique()

array(['От 1 года до 3 лет', 'От 3 до 6 лет', 'Нет опыта', 'Более 6 лет'],
      dtype=object)

In [42]:
# ВАШ КОД ЗДЕСЬ
df1 = df.copy()
df1 = df1.query("experience == 'От 3 до 6 лет' | experience == 'Нет опыта' | experience == 'От 1 года до 3 лет'")
df1 = df1.query('has_test == False')
df1 = df.sort_values('salary_to_rub', ascending=False)
df1.head(1)

,vacancy,url,created,has_test,currency,experience,schedule,skills,employer,area,description,specialization,vacancy_level,salary_from_rub,salary_to_rub
959,Back-end Developer (Python3),https://hh.ru/applicant/vacancy_response?vacan...,2021-08-02 13:21:00+03:00,False,RUR,От 1 года до 3 лет,Гибкий график,Git;Linux;Python;PostgreSQL;Django Framework;R...,Фортин,Москва,Что для нас важно: Опыт разработки на Python3...,Backend,junior,320000.0,550000.0


#### 3. Сделайте рейтинг (топ-30) навыков по всем вакансиям.

In [43]:
df.head()

,vacancy,url,created,has_test,currency,experience,schedule,skills,employer,area,description,specialization,vacancy_level,salary_from_rub,salary_to_rub
0,Backend/Full-stack developer (python),https://hh.ru/applicant/vacancy_response?vacan...,2021-08-01 13:02:48+03:00,False,RUR,От 1 года до 3 лет,Гибкий график,Python;PostgreSQL;Linux;Flask;,ATI.SU,Санкт-Петербург,Привет! Мы ATI.SU ― крупнейшая в России компан...,Fullstack,junior,120000.0,120000.0
1,Бэкенд-разработчик (Python) / Middle Python / ...,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-09 08:13:01+03:00,False,RUR,От 3 до 6 лет,Удаленная работа,Python;Git;MongoDB;Redis;Design Patterns;Flask;,"ЮТэйр, Авиакомпания",Киров (Кировская область),Utair - российская авиакомпания. Мы летаем по ...,Backend,middle,150000.0,220000.0
2,Fullstack Middle Python (Django) / VueJS Dev,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-21 15:24:16+03:00,False,RUR,От 1 года до 3 лет,Полный день,Python;Git;Django Framework;PostgreSQL;Linux;V...,КРОК,Москва,"В команду, которая занимается разработкой инст...",Fullstack,junior,175000.0,350000.0
3,Python Developer,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-28 12:57:57+03:00,False,RUR,От 3 до 6 лет,Удаленная работа,Python;Django Framework;CSS;,Mad Devs,Краснодар,Mad Devs - full stack команда для разработки и...,Python Developer (общий профиль),middle,100000.0,260000.0
4,QA Automation (Python),https://hh.ru/applicant/vacancy_response?vacan...,2021-07-31 17:10:49+03:00,False,RUR,От 1 года до 3 лет,Удаленная работа,Python;Pytest;Ansible;Selenium;,Аренадата Софтвер,Москва,"Arenadata — динамично развивающаяся компания, ...",QA/Тестирование,junior,100000.0,170000.0


In [44]:
# ВАШ КОД ЗДЕСЬ
df3 = df.copy()


list_of_skills = []

def skills_collection(data):
    list_of_skills.append(data)
    return list_of_skills

df3['skills'].apply(skills_collection)
list_of_skills

['Python;PostgreSQL;Linux;Flask;',
 'Python;Git;MongoDB;Redis;Design Patterns;Flask;',
 'Python;Git;Django Framework;PostgreSQL;Linux;Vue.js;',
 'Python;Django Framework;CSS;',
 'Python;Pytest;Ansible;Selenium;',
 'ETL;vertica;SQL;Python;Pandas;DWH;Olap (online analytical processing);СУБД;OLAP;Базы данных;MS SQL;',
 'Python;Django Framework;PostgreSQL;Linux;',
 'Python;Node.js;HTTP;Git;TCP/IP;JavaScript;PHP;C++;React.js;',
 'Python;MySQL;Базы данных;SQL;Linux;Django Framework;Flask;',
 'Python;Cloud technologies;AWS;SaaS;Git;Jenkins;Docker;Kubernetes;',
 'Python;Git;Redis;MongoDB;JavaScript;',
 'Python;Git;Linux;PostgreSQL;Django Framework;',
 'Python;',
 'Python;Git;Linux;SQL;Английский язык;',
 'Python;SQL;PostgreSQL;Consumer Goods;Golang;DWH;',
 'Python;Django Framework;MongoDB;',
 'Python;JavaScript;Git;Bootstrap;HTML5;fastapi;SQL;',
 'Python;SQL;Flask;Docker;Django Framework;',
 'Python;Django Framework;PostgreSQL;Flask;SQL;',
 'Python;PostgreSQL;SQL;Linux;Docker;asyncio;',
 'Djan

In [45]:
from collections import Counter
import re

all_skills = (';').join(list_of_skills)


all_skills_new = re.split(';', all_skills)

#удаляем пустые строки и лишние пробелы
cleaned_skills = [skill.strip() for skill in all_skills_new if skill.strip()]

#приводим к единому нижнему регистру
normalized_skills = [skill.lower() for skill in cleaned_skills]

#считаем количество каждого навыка
skills_counter = Counter(normalized_skills)

#выводим топ-30 самых частых навыков
top_skills = skills_counter.most_common(30)
top_skills

print('Топ-30 самых частых навыков:\n')
for skill, count in top_skills:
    print(f"{skill}: {count}")

Топ-30 самых частых навыков:

python: 1650
git: 622
sql: 533
postgresql: 503
django framework: 489
linux: 463
flask: 191
docker: 184
javascript: 160
английский язык: 149
mysql: 128
mongodb: 127
redis: 125
ооп: 124
rabbitmq: 72
rest: 70
pytest: 66
django: 64
css: 59
html: 59
atlassian jira: 52
c++: 50
api: 49
ms sql: 47
базы данных: 45
pandas: 44
aws: 44
celery: 43
работа в команде: 43
java: 43


#### 4. Cоставьте топ-10 городов с наибольшей средней заработной платой по вакансиям тестировщика (QA).

In [46]:
df.head()

,vacancy,url,created,has_test,currency,experience,schedule,skills,employer,area,description,specialization,vacancy_level,salary_from_rub,salary_to_rub
0,Backend/Full-stack developer (python),https://hh.ru/applicant/vacancy_response?vacan...,2021-08-01 13:02:48+03:00,False,RUR,От 1 года до 3 лет,Гибкий график,Python;PostgreSQL;Linux;Flask;,ATI.SU,Санкт-Петербург,Привет! Мы ATI.SU ― крупнейшая в России компан...,Fullstack,junior,120000.0,120000.0
1,Бэкенд-разработчик (Python) / Middle Python / ...,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-09 08:13:01+03:00,False,RUR,От 3 до 6 лет,Удаленная работа,Python;Git;MongoDB;Redis;Design Patterns;Flask;,"ЮТэйр, Авиакомпания",Киров (Кировская область),Utair - российская авиакомпания. Мы летаем по ...,Backend,middle,150000.0,220000.0
2,Fullstack Middle Python (Django) / VueJS Dev,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-21 15:24:16+03:00,False,RUR,От 1 года до 3 лет,Полный день,Python;Git;Django Framework;PostgreSQL;Linux;V...,КРОК,Москва,"В команду, которая занимается разработкой инст...",Fullstack,junior,175000.0,350000.0
3,Python Developer,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-28 12:57:57+03:00,False,RUR,От 3 до 6 лет,Удаленная работа,Python;Django Framework;CSS;,Mad Devs,Краснодар,Mad Devs - full stack команда для разработки и...,Python Developer (общий профиль),middle,100000.0,260000.0
4,QA Automation (Python),https://hh.ru/applicant/vacancy_response?vacan...,2021-07-31 17:10:49+03:00,False,RUR,От 1 года до 3 лет,Удаленная работа,Python;Pytest;Ansible;Selenium;,Аренадата Софтвер,Москва,"Arenadata — динамично развивающаяся компания, ...",QA/Тестирование,junior,100000.0,170000.0


In [47]:
# ВАШ КОД ЗДЕСЬ
area_QA_new = df.query("specialization == 'QA/Тестирование'").reset_index(drop=True)
area_QA_new.groupby('area', as_index=False) \
    .agg({'salary_to_rub': 'mean'}) \
    .rename(columns={'salary_to_rub': 'mean_salary'}) \
    .sort_values('mean_salary', ascending=False).head(10).reset_index(drop=True)

,area,mean_salary
0,Тамбов,250000.000000
1,Санкт-Петербург,220000.000000
2,Димитровград,210000.000000
3,Краснодар,210000.000000
4,Нижний Новгород,210000.000000
5,Ростов-на-Дону,210000.000000
6,Самара,210000.000000
7,Саранск,210000.000000
8,Ульяновск,210000.000000
9,Новосибирск,196666.666667


#### 5. Напишите функцию для поиска вакансий по переданным в нее значениям заработной платы, наличия тестового задания, наличия заданного текста в наименовании вакансии, наличия заданного навыка. В результате выведите количество найденных вакансий, среднюю заработную плату по найденным вакансиям и список ссылок найденных вакансий. 

In [48]:
# ВАШ КОД ЗДЕСЬ

def my_dream_job(salary, if_has_test, vacancy_text, skill_text):
    
    df5 = df.copy()
    
    #фильтруем по зарплате и наличию тестового задания
    df5 = df5[(df5['has_test'] == if_has_test) & (df5['salary_to_rub'] >= salary)]
    
    #фильтруем по вакансии - используем re.escape для экранирования спецсимволов (напр, С++)
    pattern1 = re.compile(re.escape(vacancy_text), re.IGNORECASE)
    df5 = df5[df5['vacancy'].str.contains(pattern1, na=False)]
    
    #фильтруем по скиллам - используем re.escape для экранирования спецсимволов (напр, С++)
    pattern2 = re.compile(re.escape(skill_text), re.IGNORECASE)
    df5 = df5[df5['skills'].str.contains(pattern2, na=False)]
    
    amount_of_vac = len(df5)
    mean_salary = df5['salary_to_rub'].mean()
    list_of_vac = df5['url'].to_list()
      
    return {
        'amount_of_vac': amount_of_vac,
        'mean_salary': mean_salary,
        'list_of_vac': list_of_vac
    }

my_dream_job(100000, True, 'developer', 'Python')


{'amount_of_vac': 4,
 'mean_salary': 225000.0,
 'list_of_vac': ['https://hh.ru/applicant/vacancy_response?vacancyId=46737132',
  'https://hh.ru/applicant/vacancy_response?vacancyId=46660773',
  'https://hh.ru/applicant/vacancy_response?vacancyId=46385410',
  'https://hh.ru/applicant/vacancy_response?vacancyId=46085687']}

#### *Необязательно*. Усложненный вариант - сделать атрибуты функции опциональными, чтобы иметь возможность фильтрации вакансий по любой комбинации условий.

In [49]:
# ВАШ КОД ЗДЕСЬ
#фильтрует вакансии по заданным критериям (все параметры опциональны)
    

import re
from typing import Optional, Union

def my_dream_job(
    salary: Optional[Union[int, float]] = None,
    if_has_test: Optional[bool] = None,
    vacancy_text: Optional[str] = None,
    skill_text: Optional[str] = None
):
    
    
    df_filtered = df.copy()
    
    #фильтрация по зарплате
    if salary is not None:
        df_filtered = df_filtered[df_filtered['salary_to_rub'] >= salary]
    
    #фильтрация по тестовому заданию
    if if_has_test is not None:
        df_filtered = df_filtered[df_filtered['has_test'] == if_has_test]
    
    #фильтрация по названию вакансии
    if vacancy_text:
        vacancy_pattern = re.compile(re.escape(vacancy_text), re.IGNORECASE)
        df_filtered = df_filtered[df_filtered['vacancy'].str.contains(vacancy_pattern, na=False)]
    
    #фильтрация по навыкам
    if skill_text:
        skill_pattern = re.compile(re.escape(skill_text), re.IGNORECASE)
        df_filtered = df_filtered[df_filtered['skills'].str.contains(skill_pattern, na=False)]
    
    #рассчитываем результаты
    amount_of_vac = len(df_filtered)
    mean_salary = round(df_filtered['salary_to_rub'].mean(), 2) if amount_of_vac > 0 else 0
    list_of_vac = df_filtered['url'].tolist()
    
    return {
        'amount_of_vac': amount_of_vac,
        'mean_salary': mean_salary,
        'list_of_vac': list_of_vac
    }


result = my_dream_job(salary=150000, skill_text='Python')
result

{'amount_of_vac': 1414,
 'mean_salary': 228362.95,
 'list_of_vac': ['https://hh.ru/applicant/vacancy_response?vacancyId=46106059',
  'https://hh.ru/applicant/vacancy_response?vacancyId=46420292',
  'https://hh.ru/applicant/vacancy_response?vacancyId=46593145',
  'https://hh.ru/applicant/vacancy_response?vacancyId=45905528',
  'https://hh.ru/applicant/vacancy_response?vacancyId=46158037',
  'https://hh.ru/applicant/vacancy_response?vacancyId=46030221',
  'https://hh.ru/applicant/vacancy_response?vacancyId=42905077',
  'https://hh.ru/applicant/vacancy_response?vacancyId=44778866',
  'https://hh.ru/applicant/vacancy_response?vacancyId=42608114',
  'https://hh.ru/applicant/vacancy_response?vacancyId=46160938',
  'https://hh.ru/applicant/vacancy_response?vacancyId=46444080',
  'https://hh.ru/applicant/vacancy_response?vacancyId=46052524',
  'https://hh.ru/applicant/vacancy_response?vacancyId=46259825',
  'https://hh.ru/applicant/vacancy_response?vacancyId=46614391',
  'https://hh.ru/applica

In [50]:
#сохраним этот датафрейм для построения дашборда
df.to_csv('vacancy_df.csv', index=False)